### Toy problem 1: Formulate and solve flow optimization toy problem as MINLP and QUBO

Obj Fcn: Minimize the total cost of flow <br> <br>
Constraints: <br>
&nbsp; 1. Flow conservation: The flow into each node equals the flow out of each node <br>
&nbsp; 2. At crossroads, the flow must take one of the two possible paths (binary constraint) <br>


In [1]:
import pyomo.environ as pyo
import numpy as np 

## MINLP Formulation

# Problem data 
flow_network = {'s1_s2': ('s1', 's2'), 's2_s3': ('s2', 's3'), 
                's3_s4': ('s3','s4'), 's3_s5': ('s3', 's5'), 
                's4_s6': ('s4', 's6'), 's5_s6': ('s5', 's6'),
                's6_s7': ('s6', 's7')}

flow_cost = {'s1_s2': 1, 's2_s3': 1, 's3_s4': 3, 's3_s5': 8, 's4_s6': 4, 's5_s6': 9, 's6_s7': 1}

total_flow = 10

# penalty weights for binary constraints
Ky = 10
Kz = 4

# Model
model = pyo.ConcreteModel(doc='Flow Optimization Problem')

# Sets
model.nodes = pyo.Set(initialize=['s1', 's2', 's3', 's4', 's5', 's6', 's7'], doc='nodes') 
model.edges = pyo.Set(initialize=flow_network.keys(), doc='edges')

# Decision Variables  
model.f = pyo.Var(model.edges, domain=pyo.NonNegativeReals, doc='flow on each edge')
model.y = pyo.Var(domain=pyo.Binary, doc='binary decision variable for s3 to s4 path')
model.z = pyo.Var(domain=pyo.Binary, doc='binary decision variable for s3 to s5 path')

# Parameters
model.fcost = pyo.Param(model.edges, initialize=flow_cost, doc='cost of flow on each edge', mutable=True)

# Objective Function (minimize cost)
model.totalcost = pyo.Objective(expr=sum(model.f[e]*model.fcost[e] for e in model.edges) + Ky*model.y + Kz*model.z, sense=pyo.minimize)



In [2]:

# Constraints 

# Disjunction Constraint
def disjunction_rule(model):
    return model.y + model.z <= 1
model.disjunction = pyo.Constraint(rule=disjunction_rule, doc='disjunction constraint')


# Flow Conservation Constraints
def flow_conservation_rule(model, node):
    inflow = sum(model.f[edge] for edge in model.edges if flow_network[edge][1] == node)
    outflow = sum(model.f[edge] for edge in model.edges if flow_network[edge][0] == node)
    if node == 's1':
        return outflow == total_flow  # source node
    elif node == 's7':
        return inflow == total_flow  # sink node
    else:
        return inflow == outflow  # intermediate nodes

model.flow_conservation = pyo.Constraint(model.nodes, rule=flow_conservation_rule, doc='flow conservation constraints')
'''
# Linking binary variables with flow
bigM = 1e6
smallM = 1e-6
model.crossroad1 = pyo.Constraint(expr=model.f['s3_s4'] <= model.y * bigM, doc='linking y with s3 to s4')
model.crossroad2 = pyo.Constraint(expr=model.f['s3_s5'] <= model.z * bigM, doc='linking z with s3 to s5')

# Ensure there is no flow in the path that is not chosen
model.crossroad3 = pyo.Constraint(expr=model.f['s3_s4'] >= model.y * smallM, doc='linking y with s3 to s4 lower bound')
model.crossroad4 = pyo.Constraint(expr=model.f['s3_s5'] >= model.z * smallM, doc='linking z with s3 to s5 lower bound')

'''
model.xrd1 = pyo.Constraint(expr=model.f['s3_s4'] * model.z <= 0, doc='linking z with s3 to s4')
model.xrd2 = pyo.Constraint(expr=model.f['s3_s5'] * model.y <= 0, doc='linking y with s3 to s5') 
# model.xrd3 = pyo.Constraint(expr=model.f['s3_s4'] + model.f['s3_s5'] <= model.f['s2_s3'], doc='conservation of flow at s3')

# Solve the model using a solver
solver = pyo.SolverFactory('gurobi',solver_io='python')
results = solver.solve(model, tee=True)

model.pprint()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: 13th Gen Intel(R) Core(TM) i7-1365U, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 8 rows, 9 columns and 16 nonzeros
Model fingerprint: 0xf5888fff
Model has 2 quadratic constraints


Variable types: 7 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 6 rows and 5 columns
Presolve time: 0.00s
Presolved: 6 rows, 6 columns, 12 nonzeros
Variable types: 2 continuous, 4 integer (2 binary)

Root relaxation: objective 1.000000e+02, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     100.0000000  100.00000  0.00%     -    0s

Explored 1 nodes (1 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 12 (of 12 available processors)

Solution count 1: 100 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+02, best bound 1.000000000000e+02, gap 0.0000%
Flow Optimization 